In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers

current_learning_iteration = 0

# Device Class
class Device:
    def __init__(self, device_id, ram, storage, cpu, bandwidth, battery, charging):
        self.device_id = device_id
        self.ram = ram
        self.storage = storage
        self.cpu = cpu
        self.bandwidth = bandwidth
        self.battery = battery
        self.charging = charging
        self.energy_consumption = ram + storage + cpu + bandwidth
        self.model = self.create_model()
        self.last_round_participated = 0
        self.data = None  # Placeholder for dataset partition

        self.test_data = None
        # TODO: add test data for each device
        # TODO: use the test data retrieved from load dataset from mnist and distribute it between devices and global model


        self.number_of_times_fitted = 0

    def create_model(self):
        model = keras.Sequential([
            layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dense(128, activation='relu'),
            layers.Dense(10, activation='softmax')
        ])
        model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.01),
                      loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        return model




# Global Model
# Define the global model with the same architecture
def create_global_model():
    model = keras.Sequential([
        layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.01),
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

global_model = create_global_model()



import numpy as np

def aggregate_weights(global_model, devices):
    """Computes the weighted average of model weights from all devices and updates the global model."""

    num_devices = len(devices)
    if num_devices == 0:
        print("No devices available for aggregation.")
        return

    # Get device weights and participation ratios
    device_weights = [device.model.get_weights() for device in devices]
    device_participation_ratio = np.array(
        [device.last_round_participated / current_learning_iteration for device in devices]
    )

    # Total weight for normalization
    total_weight = np.sum(device_participation_ratio)
    if total_weight == 0:
        print("Total weight is zero, cannot perform aggregation.")
        return

    for item in device_participation_ratio:
        print(item)


    len_total_devices_data = 0
    for device in devices:
        len_total_devices_data += len(device.data[0])

    # Compute weighted sum of weights
    weighted_sums = [
        np.sum(np.stack([device_weights[i][layer] * device_participation_ratio[i] * len(devices[i].data[0])/float(len_total_devices_data) for i in range(num_devices)]), axis=0)
        for layer in range(len(device_weights[0]))
    ]


    # Compute weighted average
    weighted_avg_weights = [layer_sum / total_weight for layer_sum in weighted_sums]

    # Set the global model's weights to the weighted averaged weights
    print("old global model weights:", global_model.get_weights())
    global_model.set_weights(weighted_avg_weights)
    print("new global model weights:", global_model.get_weights())
    print(np.array_equal(global_model.get_weights(), weighted_avg_weights))
    # print(global_model.get_weights())
    # for i in range(100):
    #     print()
    # print(weighted_avg_weights)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [2]:
# Load dataset from CSV
csv_file = 'devices.csv'
df = pd.read_csv(csv_file)
df.columns = df.columns.str.strip().str.lower()


# Convert CSV rows into device objects
devices = []
for _, row in df.iterrows():
    device = Device(
        row['id'], row['ram'], row['storage'], row['cpu'], row['bandwidth'], row['battery'],
        row.get('charging', 0)
    )
    devices.append(device)


# Load MNIST dataset
(x_train, y_train), (_, _) = keras.datasets.mnist.load_data()

# Normalize data and reshape for CNN
x_train = x_train.astype("float32") / 255.0
x_train = np.expand_dims(x_train, -1)  # Add channel dimension

# Shuffle data
indices = np.arange(len(x_train))
np.random.shuffle(indices)
x_train, y_train = x_train[indices], y_train[indices]

# Split into global test set (20%) and training set (80%)
split_index = int(0.8 * len(x_train))
x_train_devices, y_train_devices = x_train[:split_index], y_train[:split_index]
x_test_global, y_test_global = x_train[split_index:], y_train[split_index:]


# Split dataset among devices
num_devices = len(devices)
split_size = len(x_train_devices) // num_devices

for i, device in enumerate(devices):
    start = i * split_size
    end = (i + 1) * split_size if i < num_devices - 1 else len(x_train_devices)
    device.data = (x_train_devices[start:end], y_train_devices[start:end])


with open('bitstring.txt', 'r') as f:
    bitstring = f.read()

bitstring = [int(bit) for bit in bitstring.split(',')]

current_learning_iteration += 1
for device in devices:
    print(int(device.device_id))
    if bitstring[int(device.device_id)] == 1:
        device.model.fit(device.data[0], device.data[1], epochs=2, verbose=1)
        device.number_of_times_fitted += 1
        device.last_round_participated = current_learning_iteration


# Call this function after training the local models:
aggregate_weights(global_model, devices)


test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
print(f"Global Model Accuracy: {test_acc:.4f}")

########################################################
# # Update device participation based on the bitstring
# selected_devices = [device for device in devices if bitstring[int(device.device_id)] == 1]

# current_learning_iteration += 1
# # Train local models for selected devices
# for device in selected_devices:
#     device.model.fit(device.data[0], device.data[1], epochs=1, verbose=0)
#     device.number_of_times_fitted += 1
#     device.last_round_participated = current_learning_iteration

# # Aggregate weights to update the global model
# aggregate_weights(self.global_model, selected_devices)

# # Distribute the updated global model back to all devices
# for device in devices:
#     device.model.set_weights(self.global_model.get_weights())

# test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
# print(f"Global Model Accuracy: {test_acc:.4f}")

#################################################################

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
0
Epoch 1/2
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.1791 - loss: 2.2518
Epoch 2/2
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.5825 - loss: 1.9594
1
Epoch 1/2
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.1871 - loss: 2.2801
Epoch 2/2
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.5725 - loss: 2.0807
2
Epoch 1/2
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.2336 - loss: 2.2479
Epoch 2/2
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6218 - loss: 1.9377
3
Epoch 1/2
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.2110 - loss: 2.2479
Epoch 2/2
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.5876 - loss: 1.9259
4
Epoch 1/2
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.1789 - loss: 2.2522
Epoch 2/2
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.6223 - loss: 1.9577
5
Epoch 1/2
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.2192 - loss: 2.2422
Epoch 2/2
50/

In [3]:
!pip install pymoo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 11.5 MB/s eta 0:00:00
  Created wheel for grapheme: filename=grapheme-0.6.0-py3-none-any.whl size=210082 sha256=9e9715d9c84a68b78d64d4401222dfdeb115a23ea7d81645cb16697deb986e3d
  Stored in directory: /root/.cache/pip/wheels/ee/3b/0b/1b865800e916d671a24028d884698674138632a83fdfad4926
Successfully built grapheme


In [4]:
import numpy as np
import random
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.operators.sampling.rnd import BinaryRandomSampling
from pymoo.operators.crossover.pntx import TwoPointCrossover
from pymoo.operators.mutation.bitflip import BitflipMutation
from pymoo.operators.selection.tournament import TournamentSelection
from pymoo.termination.default import DefaultMultiObjectiveTermination

# Parameters
NUM_DEVICES = num_devices   # Number of devices (length of bitstring)
POPULATION_SIZE = 50
NUM_GENERATIONS = 3

# Step 1: Define the Problem
import numpy as np
from pymoo.core.problem import Problem

class FederatedLearningProblem(Problem):
    def __init__(self, num_devices, devices, global_model, x_test_global, y_test_global):
        super().__init__(
            n_var=num_devices,         # Number of variables (bitstring length)
            n_obj=3,                   # Number of objectives
            n_constr=0,                # No constraints
            xl=np.zeros(num_devices),  # Lower bound (0)
            xu=np.ones(num_devices),   # Upper bound (1)
            type_var=np.bool_          # Binary variables (bitstrings)
        )
        self.devices = devices
        self.global_model = global_model
        self.x_test_global = x_test_global
        self.y_test_global = y_test_global

        # Save the initial global model weights
        self.initial_global_weights = global_model.get_weights()

    def _evaluate(self, X, out, *args, **kwargs):
        """Evaluates objective values for each solution in the population."""
        num_solutions = len(X)
        F = np.zeros((num_solutions, 3))  # Initialize objective matrix

        for i, bitstring in enumerate(X):
            # Reset the global model to its initial state
            self.global_model.set_weights(self.initial_global_weights)
            # Update device participation based on the bitstring
            selected_devices = [device for device, bit in zip(self.devices, bitstring) if bit == 1]
            # Objective 1: Hardware Objectives (maximize)
            hardware_score = sum(
                device.ram + device.storage + device.cpu + device.bandwidth + device.battery + device.charging
                for device in selected_devices
            )
            F[i, 0] = ((6 - hardware_score)/6.0)  # Minimize (negative of hardware score)
            # Objective 2: Fairness (prioritize devices with lowest local accuracy)
            local_accuracies = []
            for device in self.devices:
              # just the devices in this solution:
                if bitstring[int(device.device_id)] == 1:
                  _, accuracy = global_model.evaluate(device.data[0], device.data[1], verbose=0)
                  local_accuracies.append(accuracy)
                else:
                  local_accuracies.append(0)
            # Fairness score: Sum of (1 - accuracy) for selected devices should be minimized
            # This prioritizes devices with lower local accuracy

            fairness_score = 0  # Initialize fairness score

            for j, bit in enumerate(bitstring):
                # print(j)
                # print(bit)
                if bit == 1:  # Check if the device is selected
                    fairness_score += local_accuracies[j]  # Add the corresponding accuracy value
            # Now, fairness_score contains the final sum

            # fairness_score = sum(local_accuracies[j] for j, bit in enumerate(bitstring) if bit == 1)
            F[i, 1] = fairness_score/float(len(selected_devices))  # Minimize (negative of fairness score)  # Added (/Selected Devices) to normalize between 0 and 1
            # Objective 3: Global Model Accuracy (maximize)
            _, global_accuracy = self.global_model.evaluate(self.x_test_global, self.y_test_global, verbose=0)
            F[i, 2] = 1 - global_accuracy  # Minimize (1 - accuracy)
        out["F"] = F  # Set the objective values

'''
            # Update device participation based on the bitstring
            selected_devices = [device for device, bit in zip(self.devices, bitstring) if bit == 1]

            # Train local models for selected devices
            for device in selected_devices:
                device.model.fit(device.data[0], device.data[1], epochs=1, verbose=0)

            # Aggregate weights to update the global model
            aggregate_weights(self.global_model, selected_devices)

            # Distribute the updated global model back to all devices
            for device in self.devices:
                device.model.set_weights(self.global_model.get_weights())
            '''

'\n            # Update device participation based on the bitstring\n            selected_devices = [device for device, bit in zip(self.devices, bitstring) if bit == 1]\n\n            # Train local models for selected devices\n            for device in selected_devices:\n                device.model.fit(device.data[0], device.data[1], epochs=1, verbose=0)\n\n            # Aggregate weights to update the global model\n            aggregate_weights(self.global_model, selected_devices)\n\n            # Distribute the updated global model back to all devices\n            for device in self.devices:\n                device.model.set_weights(self.global_model.get_weights())\n            '

In [5]:


problem = FederatedLearningProblem(
    num_devices=NUM_DEVICES,
    devices=devices,
    global_model=global_model,
    x_test_global=x_test_global,
    y_test_global=y_test_global
)


# Step 2: Configure NSGA-II Algorithm
algorithm = NSGA2(
    pop_size=POPULATION_SIZE,
    sampling=BinaryRandomSampling(),      # Random bitstrings
    crossover=TwoPointCrossover(),        # Two-point crossover
    mutation=BitflipMutation(),           # Bit flip mutation
    eliminate_duplicates=True             # Avoid duplicate solutions
)


NUMBER_OF_LEARNING_ITERATIONS = 2

In [6]:
# Step 3: Run Optimization
res = minimize(
    problem=problem,
    algorithm=algorithm,
    # termination=MultiObjectiveSpaceToleranceTermination(tol=1e-6, n_last=10, nth_gen=5, n_max_gen=NUM_GENERATIONS),
    termination=DefaultMultiObjectiveTermination(n_max_gen=NUM_GENERATIONS),
    # seed=42,
    verbose=True
)

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |       50 |      7 |             - |             -
     2 |      100 |      8 |  0.0518273986 |             f
     3 |      150 |      9 |  0.2006384358 |         ideal


In [7]:
# Step 4: Extract the Best Pareto Front
pareto_front = res.F   # Objective values of solutions in Pareto front
pareto_solutions = res.X  # Corresponding bitstrings

# Print the Best Pareto Front Solutions
print("Best Pareto Front (Bitstrings):")
for bitstring in pareto_solutions:
    print("".join(map(str, bitstring)).replace('True','1').replace('False','0'))

bitstring = pareto_solutions[0] # for now!
bitstring = str(bitstring).replace('False','0').replace('True','1')
for char in bitstring:
    if char != '0' and char != '1':
        bitstring = bitstring.replace(char,'')

print(len(bitstring))
print(bitstring)
temp_bitstring = []
for bit in bitstring:
    temp_bitstring.append(bit)
bitstring = temp_bitstring

########################################################
# Update device participation based on the bitstring
selected_devices = [device for device in devices if bitstring[int(device.device_id)] == '1']

# Aggregate weights to update the global model
# aggregate_weights(global_model, selected_devices)
aggregate_weights(global_model, devices)

# Distribute the updated global model back to all devices
for device in devices:
    device.model.set_weights(global_model.get_weights())

current_learning_iteration += 1
# Train local models for selected devices
for device in selected_devices:
    device.model.fit(device.data[0], device.data[1], epochs=5, verbose=0)
    device.number_of_times_fitted += 1
    device.last_round_participated = current_learning_iteration


test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
print(f"Global Model Accuracy: {test_acc:.4f}")

Best Pareto Front (Bitstrings):
000000111011010010111011100100
010001111010111110111001011111
111100101111110101111111011100
101011010111111111101110011111
011110101111110101011111111100
001011101111110111011111011101
100001001111111111111011111100
101001111010111110111001011110
000101011100100110111001000100
30
000000111011010010111011100100
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
old global model weights: [array([[[[ 2.51962134e-04,  1.14540011e-03,  2.84619164e-04,
           1.54530804e-03,  7.29253748e-04, -3.26965732e-04,
           1.52899476e-04, -9.64046922e-06,  6.67585467e-04,
           1.56498188e-03,  7.19902746e-04,  5.34691033e-04,
           7.36815215e-04,  1.34074508e-04,  6.79416407e-05,
          -3.50243994e-04,  6.56167162e-04,  6.66481734e-04,
           6.52362243e-04,  1.28061895e-03, -3.35621909e-04,
          -1.06965148e-04,  4.31599532e-04,  4.06062405e-04,
           3.7652551

In [8]:
print(bitstring) # for now!
print(len(bitstring))
print(bitstring.count("1"))

['0', '0', '0', '0', '0', '0', '1', '1', '1', '0', '1', '1', '0', '1', '0', '0', '1', '0', '1', '1', '1', '0', '1', '1', '1', '0', '0', '1', '0', '0']
30
14


In [9]:
# Step 3: Run Optimization
problem.initial_global_weights = global_model.get_weights()
res = minimize(
    problem=problem,
    algorithm=algorithm,
    termination=DefaultMultiObjectiveTermination(n_max_gen=NUM_GENERATIONS),
    # seed=42,
    verbose=True
)

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |       50 |      5 |             - |             -
     2 |      100 |      6 |  0.0893156216 |         ideal
     3 |      150 |      8 |  0.0231671355 |         ideal


In [10]:
########################################################
# Update device participation based on the bitstring
selected_devices = [device for device in devices if bitstring[int(device.device_id)] == '1']

# aggregate_weights(global_model, selected_devices)
aggregate_weights(global_model, devices)
# Distribute the updated global model back to all devices
for device in devices:
    device.model.set_weights(global_model.get_weights())

current_learning_iteration += 1
# Train local models for selected devices
for device in selected_devices:
    device.number_of_times_fitted += 1
    device.last_round_participated = current_learning_iteration
    loss_before = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    device.model.fit(device.data[0], device.data[1], epochs=10, verbose=0)
    loss_after = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    print(f"Device {device.device_id}: Loss before: {loss_before}, Loss after: {loss_after}")

# Aggregate weights to update the global model
w1 = global_model.get_weights()
aggregate_weights(global_model, selected_devices)
w2 = global_model.get_weights()
print("global model stayed the same?")
print(np.array_equal(w1,w2))
test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
print(f"Global Model Accuracy: {test_acc:.4f}")

0.5
0.5
0.5
0.5
0.5
0.5
1.0
1.0
1.0
0.5
1.0
1.0
0.5
1.0
0.5
0.5
1.0
0.5
1.0
1.0
1.0
0.5
1.0
1.0
1.0
0.5
0.5
1.0
0.5
0.5
old global model weights: [array([[[[ 2.51962134e-04,  1.14540011e-03,  2.84619164e-04,
           1.54530804e-03,  7.29253748e-04, -3.26965732e-04,
           1.52899476e-04, -9.64046922e-06,  6.67585467e-04,
           1.56498188e-03,  7.19902746e-04,  5.34691033e-04,
           7.36815215e-04,  1.34074508e-04,  6.79416407e-05,
          -3.50243994e-04,  6.56167162e-04,  6.66481734e-04,
           6.52362243e-04,  1.28061895e-03, -3.35621909e-04,
          -1.06965148e-04,  4.31599532e-04,  4.06062405e-04,
           3.76525510e-04,  1.03858463e-03,  2.60709116e-04,
           8.15543695e-04,  5.73920610e-04,  3.36067402e-04,
          -2.13378575e-04, -3.49384354e-04]],

        [[ 6.96825038e-04,  4.23317746e-04,  1.64897181e-04,
           2.51976744e-04,  5.96566882e-04,  8.27675918e-04,
           8.99634266e-04, -2.55671068e-04,  4.30489832e-04,
           1.

In [11]:
# Step 3: Run Optimization
problem.initial_global_weights = global_model.get_weights()
res = minimize(
    problem=problem,
    algorithm=algorithm,
    termination=DefaultMultiObjectiveTermination(n_max_gen=NUM_GENERATIONS),
    # seed=42,
    verbose=True
)########################################################
# Update device participation based on the bitstring
selected_devices = [device for device in devices if bitstring[int(device.device_id)] == '1']

# aggregate_weights(global_model, selected_devices)
aggregate_weights(global_model, devices)
# Distribute the updated global model back to all devices
for device in devices:
    device.model.set_weights(global_model.get_weights())

current_learning_iteration += 1
# Train local models for selected devices
for device in selected_devices:
    device.number_of_times_fitted += 1
    device.last_round_participated = current_learning_iteration
    loss_before = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    device.model.fit(device.data[0], device.data[1], epochs=10, verbose=0)
    loss_after = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    print(f"Device {device.device_id}: Loss before: {loss_before}, Loss after: {loss_after}")

# Aggregate weights to update the global model
w1 = global_model.get_weights()
aggregate_weights(global_model, selected_devices)
w2 = global_model.get_weights()
print("global model stayed the same?")
print(np.array_equal(w1,w2))
test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
print(f"Global Model Accuracy: {test_acc:.4f}")

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |       50 |      7 |             - |             -
     2 |      100 |      8 |  0.3353474663 |         ideal
     3 |      150 |      9 |  0.1098736297 |         ideal
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
1.0
1.0
1.0
0.3333333333333333
1.0
1.0
0.3333333333333333
1.0
0.3333333333333333
0.3333333333333333
1.0
0.3333333333333333
1.0
1.0
1.0
0.3333333333333333
1.0
1.0
1.0
0.3333333333333333
0.3333333333333333
1.0
0.3333333333333333
0.3333333333333333
old global model weights: [array([[[[ 6.0380592e-07,  2.7357471e-06,  6.8484053e-07,
           3.6843189e-06,  1.7437918e-06, -7.7203816e-07,
           3.6990465e-07, -1.7753981e-08,  1.5965969e-06,
           3.7381762e-06,  1.7191084e-06,  1.2848515e-06,
           1.7654572e-06,  3.2618740e-07,  1.6976374e-07,
          -8.3308123e-07,  1.5677068e-06,  1.5967752e-06,
           1.5637795e-06,  3.

In [12]:
# Step 3: Run Optimization
problem.initial_global_weights = global_model.get_weights()
res = minimize(
    problem=problem,
    algorithm=algorithm,
    termination=DefaultMultiObjectiveTermination(n_max_gen=NUM_GENERATIONS),
    # seed=42,
    verbose=True
)########################################################
# Update device participation based on the bitstring
selected_devices = [device for device in devices if bitstring[int(device.device_id)] == '1']

# aggregate_weights(global_model, selected_devices)
aggregate_weights(global_model, devices)
# Distribute the updated global model back to all devices
for device in devices:
    device.model.set_weights(global_model.get_weights())

current_learning_iteration += 1
# Train local models for selected devices
for device in selected_devices:
    device.number_of_times_fitted += 1
    device.last_round_participated = current_learning_iteration
    loss_before = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    device.model.fit(device.data[0], device.data[1], epochs=10, verbose=0)
    loss_after = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    print(f"Device {device.device_id}: Loss before: {loss_before}, Loss after: {loss_after}")

# Aggregate weights to update the global model
w1 = global_model.get_weights()
aggregate_weights(global_model, selected_devices)
w2 = global_model.get_weights()
print("global model stayed the same?")
print(np.array_equal(w1,w2))
test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
print(f"Global Model Accuracy: {test_acc:.4f}")

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |       50 |      6 |             - |             -
     2 |      100 |      8 |  0.1224488786 |         ideal
     3 |      150 |     12 |  0.2925845869 |         ideal
0.25
0.25
0.25
0.25
0.25
0.25
1.0
1.0
1.0
0.25
1.0
1.0
0.25
1.0
0.25
0.25
1.0
0.25
1.0
1.0
1.0
0.25
1.0
1.0
1.0
0.25
0.25
1.0
0.25
0.25
old global model weights: [array([[[[ 2.01239967e-08,  9.11851785e-08,  2.28224533e-08,
           1.22806256e-07,  5.81209498e-08, -2.57392649e-08,
           1.23259669e-08, -5.95526906e-10,  5.32144995e-08,
           1.24595871e-07,  5.72999106e-08,  4.28197602e-08,
           5.88404170e-08,  1.08675255e-08,  5.65270142e-09,
          -2.77697598e-08,  5.22528190e-08,  5.32184892e-08,
           5.21183630e-08,  1.01831070e-07, -2.65405049e-08,
          -8.22140400e-09,  3.44195463e-08,  3.23963434e-08,
           3.00991800e-08,  8.26621473e-08,  2.08911342e-08,
           6.48186571e-08,  4.56144917e-08,  2.68066

In [ ]:
# Step 3: Run Optimization
problem.initial_global_weights = global_model.get_weights()
res = minimize(
    problem=problem,
    algorithm=algorithm,
    termination=DefaultMultiObjectiveTermination(n_max_gen=NUM_GENERATIONS),
    # seed=42,
    verbose=True
)########################################################
# Update device participation based on the bitstring
selected_devices = [device for device in devices if bitstring[int(device.device_id)] == '1']

# aggregate_weights(global_model, selected_devices)
aggregate_weights(global_model, devices)
# Distribute the updated global model back to all devices
for device in devices:
    device.model.set_weights(global_model.get_weights())

current_learning_iteration += 1
# Train local models for selected devices
for device in selected_devices:
    device.number_of_times_fitted += 1
    device.last_round_participated = current_learning_iteration
    loss_before = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    device.model.fit(device.data[0], device.data[1], epochs=10, verbose=0)
    loss_after = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    print(f"Device {device.device_id}: Loss before: {loss_before}, Loss after: {loss_after}")

# Aggregate weights to update the global model
w1 = global_model.get_weights()
aggregate_weights(global_model, selected_devices)
w2 = global_model.get_weights()
print("global model stayed the same?")
print(np.array_equal(w1,w2))
test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
print(f"Global Model Accuracy: {test_acc:.4f}")

In [ ]:
# ########################################################
# # Update device participation based on the bitstring
# selected_devices = [device for device in devices if bitstring[int(device.device_id)] == '1']

# # Aggregate weights to update the global model
# aggregate_weights(global_model, selected_devices)

# # Distribute the updated global model back to all devices
# for device in devices:
#     device.model.set_weights(global_model.get_weights())

# current_learning_iteration += 1
# # Train local models for selected devices
# for device in selected_devices:
#     device.model.fit(device.data[0], device.data[1], epochs=5, verbose=0)
#     device.number_of_times_fitted += 1
#     device.last_round_participated = current_learning_iteration

# # Aggregate weights to update the global model
# aggregate_weights(global_model, selected_devices)

# test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
# print(f"Global Model Accuracy: {test_acc:.4f}")

In [ ]:
# ########################################################
# # Update device participation based on the bitstring
# selected_devices = [device for device in devices if bitstring[int(device.device_id)] == '1']

# aggregate_weights(global_model, selected_devices)
# # Distribute the updated global model back to all devices
# for device in devices:
#     device.model.set_weights(global_model.get_weights())

# current_learning_iteration += 1
# # Train local models for selected devices
# for device in selected_devices:
#     device.model.fit(device.data[0], device.data[1], epochs=10, verbose=0)
#     device.number_of_times_fitted += 1
#     device.last_round_participated = current_learning_iteration

# # Aggregate weights to update the global model
# w1 = global_model.get_weights()
# aggregate_weights(global_model, selected_devices)
# w2 = global_model.get_weights()
# print("global model stayed the same?")
# print(np.array_equal(w1,w2))
# test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
# print(f"Global Model Accuracy: {test_acc:.4f}")

In [ ]:
# print("Before aggregation:", w1[0].flatten()[:5])
# aggregate_weights(global_model, selected_devices)
# print("After aggregation:", global_model.get_weights()[0].flatten()[:5])

In [ ]:
# ########################################################
# # Update device participation based on the bitstring
# selected_devices = [device for device in devices if bitstring[int(device.device_id)] == '1']

# aggregate_weights(global_model, selected_devices)
# # Distribute the updated global model back to all devices
# for device in devices:
#     device.model.set_weights(global_model.get_weights())

# current_learning_iteration += 1
# # Train local models for selected devices
# for device in selected_devices:
#     device.number_of_times_fitted += 1
#     device.last_round_participated = current_learning_iteration
#     loss_before = device.model.evaluate(device.data[0], device.data[1], verbose=0)
#     device.model.fit(device.data[0], device.data[1], epochs=10, verbose=0)
#     loss_after = device.model.evaluate(device.data[0], device.data[1], verbose=0)
#     print(f"Device {device.device_id}: Loss before: {loss_before}, Loss after: {loss_after}")

# # Aggregate weights to update the global model
# w1 = global_model.get_weights()
# aggregate_weights(global_model, selected_devices)
# w2 = global_model.get_weights()
# print("global model stayed the same?")
# print(np.array_equal(w1,w2))
# test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
# print(f"Global Model Accuracy: {test_acc:.4f}")

In [ ]:
# for i in range(NUMBER_OF_LEARNING_ITERATIONS):
#     # Step 3: Run Optimization
#     res = minimize(
#         problem=problem,
#         algorithm=algorithm,
#         # termination=MultiObjectiveSpaceToleranceTermination(tol=1e-6, n_last=10, nth_gen=5, n_max_gen=NUM_GENERATIONS),
#         termination=DefaultMultiObjectiveTermination(n_max_gen=NUM_GENERATIONS),
#         seed=42,
#         verbose=True
#     )


#     # Step 4: Extract the Best Pareto Front
#     pareto_front = res.F   # Objective values of solutions in Pareto front
#     pareto_solutions = res.X  # Corresponding bitstrings

#     # Print the Best Pareto Front Solutions
#     print("Best Pareto Front (Bitstrings):")
#     for bitstring in pareto_solutions:
#         print("".join(map(str, bitstring)))

#     bitstring = pareto_solutions[0] # for now!
#     print(bitstring)
#     # Convert to bitstring (list of 1s and 0s)
#     bitstring = [1 if word == "True" else 0 for word in str(bitstring).replace("True", "1 ").replace("False", "0 ").split()]


#     ########################################################
#     # Update device participation based on the bitstring
#     selected_devices = [device for device in devices if bitstring[int(device.device_id)] == 1]

#     # Aggregate weights to update the global model
#     aggregate_weights(global_model, selected_devices)

#     # Distribute the updated global model back to all devices
#     for device in devices:
#         device.model.set_weights(global_model.get_weights())

#     current_learning_iteration += 1
#     # Train local models for selected devices
#     for device in selected_devices:
#         device.model.fit(device.data[0], device.data[1], epochs=1, verbose=0)
#         device.number_of_times_fitted += 1
#         device.last_round_participated = current_learning_iteration


#     test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
#     print(f"Global Model Accuracy: {test_acc:.4f}")

#     ################################################################